# Performance Metrics and Visualization

This notebook computes and visualizes performance metrics for equity investment strategies.

## Introduction

This analysis evaluates multiple equity strategies by calculating key performance indicators including:
- Total Return
- Annualized Return
- Volatility
- Sharpe Ratio
- Maximum Drawdown

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

## Methodology

### Data Preparation
Load and prepare returns data for analysis. Ensure data is properly formatted with datetime index.

In [ ]:
# Load returns data
# returns = pd.read_csv('../data/strategy_returns.csv', index_col=0, parse_dates=True)

# Configuration
annual_factor = 252  # Trading days per year
annual_rf = 0.02  # Annual risk-free rate

### Metric Calculation Functions

Define functions to compute standard performance metrics.

In [ ]:
def compute_cumulative(returns_series):
    """Compute cumulative returns from a series of returns."""
    return (1 + returns_series).cumprod()

def annualized_return(returns_series, annual_factor):
    """Calculate annualized return."""
    cum_return = compute_cumulative(returns_series).iloc[-1]
    n_periods = len(returns_series.dropna())
    if n_periods == 0:
        return np.nan
    return (cum_return ** (annual_factor / n_periods)) - 1

def volatility(returns_series, annual_factor):
    """Calculate annualized volatility."""
    return returns_series.std() * np.sqrt(annual_factor)

def sharpe(returns_series, annual_factor, annual_rf):
    """Calculate Sharpe ratio."""
    ann_return = annualized_return(returns_series, annual_factor)
    vol = volatility(returns_series, annual_factor)
    if vol == 0 or np.isnan(vol):
        return np.nan
    return (ann_return - annual_rf) / vol

def max_drawdown(returns_series):
    """Calculate maximum drawdown."""
    cum = compute_cumulative(returns_series)
    peak = cum.cummax()
    dd = (cum / peak) - 1
    return dd.min()

## Performance Metrics Computation

Calculate metrics for each strategy and create summary DataFrame.

In [ ]:
# Compute metrics per strategy
# metrics = {}
# for col in returns.columns:
#     ser = returns[col]
#     metrics[col] = {
#         'Total Return': compute_cumulative(ser).iloc[-1] - 1 if not ser.dropna().empty else np.nan,
#         'Annualized Return': annualized_return(ser, annual_factor),
#         'Volatility': volatility(ser, annual_factor),
#         'Sharpe': sharpe(ser, annual_factor, annual_rf),
#         'Max Drawdown': max_drawdown(ser)
#     }
# 
# metrics_df = pd.DataFrame(metrics).T
# metrics_df = metrics_df[['Total Return', 'Annualized Return', 'Volatility', 'Sharpe', 'Max Drawdown']]
# metrics_df.sort_values('Sharpe', ascending=False, inplace=True)
# metrics_df

## Manager Summary Tables

Tables below provide clean, labeled summaries. Percentages shown where applicable.

In [ ]:
# Format metrics for presentation
# fmt_df = metrics_df.copy()
# pct_cols = ['Total Return', 'Annualized Return', 'Volatility', 'Max Drawdown']
# for c in pct_cols:
#     if c in fmt_df:
#         fmt_df[c] = (fmt_df[c] * 100).map(lambda x: f'{x:,.2f}%' if pd.notna(x) else '')
# fmt_df['Sharpe'] = fmt_df['Sharpe'].map(lambda x: f'{x:,.2f}' if pd.notna(x) else '')
# fmt_df

## Comparative Visualizations

Visualize performance metrics across strategies for easy comparison.

In [ ]:
# Create bar charts for key metrics
# plot_df = metrics_df.copy()
# fig, axes = plt.subplots(2, 2, figsize=(14, 10))
# 
# # Annualized Return
# plot_df['Annualized Return'].plot(kind='bar', ax=axes[0,0], color='C0')
# axes[0,0].set_title('Annualized Return')
# axes[0,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
# 
# # Volatility
# plot_df['Volatility'].plot(kind='bar', ax=axes[0,1], color='C1')
# axes[0,1].set_title('Volatility')
# axes[0,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
# 
# # Sharpe Ratio
# plot_df['Sharpe'].plot(kind='bar', ax=axes[1,0], color='C2')
# axes[1,0].set_title('Sharpe Ratio')
# 
# # Max Drawdown
# plot_df['Max Drawdown'].plot(kind='bar', ax=axes[1,1], color='C3')
# axes[1,1].set_title('Maximum Drawdown')
# axes[1,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
# 
# plt.tight_layout()
# plt.show()

## Manager Guidance

### How to Use This Notebook

1. **Load Data**: Uncomment the data loading section and point to your returns CSV file
2. **Run Analysis**: Execute all cells to compute metrics and generate visualizations
3. **Interpret Results**: Review the formatted tables and charts to compare strategy performance
4. **Customize**: Adjust the annual_factor and annual_rf parameters as needed for your analysis

### Key Metrics Explained

- **Total Return**: Cumulative return over the entire period
- **Annualized Return**: Geometric average return per year
- **Volatility**: Standard deviation of returns (annualized)
- **Sharpe Ratio**: Risk-adjusted return (higher is better)
- **Max Drawdown**: Largest peak-to-trough decline (lower is better)